In [1]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer, BitsAndBytesConfig, DataCollatorForSeq2Seq
from peft import PeftModel, PeftConfig
from datasets import load_dataset, concatenate_datasets

from metrics import compute_metrics
from process_data import build_process_fn
from custom_trainer import WeightedLossT5
from config import SEPARATOR

# Lower precision for higher performance (negligible impact on results)
torch.set_float32_matmul_precision('high')

DATA_DOCUMENTS = "/home/nub/Bachelor/bachelor-thesis/data/processed/documents.csv"
DATA_DOCUMENTS_AUG = "/home/nub/Bachelor/bachelor-thesis/data/processed/documents_aug.csv"
DATA_TRAIN_PROC = "/home/nub/Bachelor/bachelor-thesis/data/processed/train.csv"
DATA_EVAL_PROC = "/home/nub/Bachelor/bachelor-thesis/data/processed/eval.csv"
DATA_TEST_PROC = "/home/nub/Bachelor/bachelor-thesis/data/processed/test.csv"
MODELS_DIR = "/home/nub/Bachelor/bachelor-thesis/models"

num_cpus = 16

# Enable debug to drastically reduce values
DEBUG = False
DEBUG_SIZE = 4
SPLITS = ["train", "eval", "test"]

USE_COT = False
USE_AUG = False
USE_INDEXING = False

SAVE_DIR = "/home/nub/Bachelor/bachelor-thesis/models/finqa_full_base_low"


peft_model_path = SAVE_DIR
peft_config = PeftConfig.from_pretrained(peft_model_path)
base_model_name = peft_config.base_model_name_or_path

tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    cache_dir=MODELS_DIR,
    local_files_only=True,
    use_fast=True,
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

base_model = WeightedLossT5.from_pretrained(
    base_model_name,
    cache_dir=MODELS_DIR,
    local_files_only=True,
    quantization_config=bnb_config,
    device_map="auto",
)
base_model.tokenizer = tokenizer
base_model.use_cot = USE_COT

model = PeftModel.from_pretrained(base_model, peft_model_path)


In [2]:
# Process documents for indexing
if USE_AUG:
    # Augmented documents use pseudo-queries which should use the retrieval task instead
    input_key = "pseudo_query"
    document_task = False
    document_file = DATA_DOCUMENTS_AUG
else:
    # Traditional documents should use the indexing task
    input_key = "document"
    document_task = True
    document_file = DATA_DOCUMENTS

if USE_INDEXING:
    raw_documents_ds = load_dataset("csv", data_files=document_file, split="train")
    documents_ds = raw_documents_ds.map(
        build_process_fn(tokenizer, input_key, document_task, USE_COT),
        remove_columns=raw_documents_ds.column_names,
        num_proc=num_cpus,
        batched=True,
        batch_size=max(1, len(raw_documents_ds) // num_cpus)
    )

# Process data for retrieval (train, valid, test)
file_mapping = {
    "train": DATA_TRAIN_PROC,
    "eval": DATA_EVAL_PROC,
    "test": DATA_TEST_PROC,
}

# Process queries for retrieval
raw_data_ds = load_dataset("csv", data_files=file_mapping)
tokenized_ds = raw_data_ds.map(
    build_process_fn(tokenizer, "question", False, USE_COT),
    remove_columns=raw_data_ds["train"].column_names,
    num_proc=num_cpus,
    batched=True,
    batch_size=max(1, len(raw_data_ds["eval"]) // num_cpus)
)

# Merge the indexing stage into the train split (overriden by sampling)
if USE_INDEXING:
    tokenized_ds["train"] = concatenate_datasets([tokenized_ds["train"], documents_ds])

# Reduce data size for all splits
if DEBUG:
    raw_documents_ds = raw_documents_ds.select(range(DEBUG_SIZE))

    for split in SPLITS:
        tokenized_ds[split] = tokenized_ds[split].select(range(DEBUG_SIZE))
        raw_data_ds[split] = raw_data_ds[split].select(range(DEBUG_SIZE))

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [3]:
def evaluate_model(model, dataset):
    model.eval()
    dataloader = DataLoader(
        dataset,
        batch_size=32,
        collate_fn=data_collator,
        shuffle=False,
        drop_last=False,
        num_workers=0 if DEBUG else num_cpus,
        prefetch_factor=None if DEBUG else 2,
        pin_memory=False if DEBUG else True,
    )

    all_preds = []
    all_labels = []

    for batch in tqdm(dataloader):
        input_ids = batch["input_ids"].to(model.device)
        attention_mask = batch["attention_mask"].to(model.device)
        labels = batch["labels"].to(model.device)

        with torch.no_grad():
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                max_new_tokens=128,
                num_beams=10,
                early_stopping=True,
            )
        
        decoded_preds = model.base_model.decode_tokens(generated_ids.detach().cpu().numpy())
        decoded_labels = model.base_model.decode_tokens(labels.detach().cpu().numpy())

        all_preds.extend(decoded_preds)
        all_labels.extend(decoded_labels)
    
    metrics, _ = compute_metrics(all_preds, all_labels, USE_COT)
    
    return metrics

results = {}

for split in SPLITS:
    print(f"Evaluating split: {split}")
    results[split] = {
        **evaluate_model(model, tokenized_ds[split])
    }

print(results)

Evaluating split: train


100%|██████████| 196/196 [06:24<00:00,  1.96s/it]


Evaluating split: eval


100%|██████████| 28/28 [00:54<00:00,  1.94s/it]


Evaluating split: test


100%|██████████| 36/36 [01:10<00:00,  1.95s/it]

{'train': {'penalty_scaled': np.float64(1.0816701327787555), 'penalty_capped': np.float64(1.0816701327787555), 'penalty_uncapped': np.float64(1.0857974724044153), 'missing': 0.0, 'exact_match_accuracy': 0.9054551271796513, 'part_match_accuracy': 0.9445155441795946, 'set_match_accuracy': 0.9493840985442297, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0}, 'eval': {'penalty_scaled': np.float64(1.4816459041147605), 'penalty_capped': np.float64(1.4816459041147605), 'penalty_uncapped': np.float64(1.5104643261608155), 'missing': 0.0, 'exact_match_accuracy': 0.4824462061155153, 'part_match_accuracy': 0.6753491883729695, 'set_match_accuracy': 0.6946772366930908, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0}, 'test': {'penalty_scaled': np.float64(1.4557221737866899), 'penalty_capped': np.float64(1.4557221737866899), 'penalty_uncapped': np.float64(1.4821505376344086), 'missing': 0.0, 'exact_match_accuracy': 0.50479511769

/home/nub/Bachelor/bachelor-thesis/models/finqa_full_base_pseudo

{'train': {'penalty_scaled': np.float64(1.0127699568069108), 'penalty_capped': np.float64(1.0127699568069108), 'penalty_uncapped': np.float64(1.0127699568069108), 'missing': 0.0, 'exact_match_accuracy': 0.9705647096464566, 'part_match_accuracy': 0.9849357436143541, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0},

'eval': {'penalty_scaled': np.float64(1.2865515288788223), 'penalty_capped': np.float64(1.2865515288788223), 'penalty_uncapped': np.float64(1.2865515288788223), 'missing': 0.0, 'exact_match_accuracy': 0.45300113250283125, 'part_match_accuracy': 0.6543978859947137, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0},

'test': {'penalty_scaled': np.float64(1.2886878814298168), 'penalty_capped': np.float64(1.2886878814298168), 'penalty_uncapped': np.float64(1.2886878814298168), 'missing': 0.0, 'exact_match_accuracy': 0.44638186573670446, 'part_match_accuracy': 0.6519907003778002, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0}}

/home/nub/Bachelor/bachelor-thesis/models/finqa_full_base_prompt

{'train': {'penalty_scaled': np.float64(1.1170916653335468), 'penalty_capped': np.float64(1.1170916653335468), 'penalty_uncapped': np.float64(1.1170916653335468), 'missing': 0.0, 'exact_match_accuracy': 0.8341065429531275, 'part_match_accuracy': 0.8994827494267531, 'set_match_accuracy': 0.908041380045855, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0},

'eval': {'penalty_scaled': np.float64(1.3450887127217819), 'penalty_capped': np.float64(1.3450887127217819), 'penalty_uncapped': np.float64(1.3450887127217819), 'missing': 0.0, 'exact_match_accuracy': 0.5209513023782559, 'part_match_accuracy': 0.7095130237825605, 'set_match_accuracy': 0.7258588146470372, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0},

'test': {'penalty_scaled': np.float64(1.3277361232199942), 'penalty_capped': np.float64(1.3277361232199942), 'penalty_uncapped': np.float64(1.3277361232199942), 'missing': 0.0, 'exact_match_accuracy': 0.5527462946817786, 'part_match_accuracy': 0.7213019471084022, 'set_match_accuracy': 0.740860215053766, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0}}

/home/nub/Bachelor/bachelor-thesis/models/finqa_full_base_low

{'train': {'penalty_scaled': np.float64(1.0816701327787555), 'penalty_capped': np.float64(1.0816701327787555), 'penalty_uncapped': np.float64(1.0857974724044153), 'missing': 0.0, 'exact_match_accuracy': 0.9054551271796513, 'part_match_accuracy': 0.9445155441795946, 'set_match_accuracy': 0.9493840985442297, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0},

'eval': {'penalty_scaled': np.float64(1.4816459041147605), 'penalty_capped': np.float64(1.4816459041147605), 'penalty_uncapped': np.float64(1.5104643261608155), 'missing': 0.0, 'exact_match_accuracy': 0.4824462061155153, 'part_match_accuracy': 0.6753491883729695, 'set_match_accuracy': 0.6946772366930908, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0},

'test': {'penalty_scaled': np.float64(1.4557221737866899), 'penalty_capped': np.float64(1.4557221737866899), 'penalty_uncapped': np.float64(1.4821505376344086), 'missing': 0.0, 'exact_match_accuracy': 0.5047951176983435, 'part_match_accuracy': 0.6887532693984343, 'set_match_accuracy': 0.7145015983725687, 'structure_score_norm': 0.0, 'structure_score_pos': 0.0, 'structure_score_neg': 0.0}}